In [ ]:
from tensorflow import keras

In [1]:
from pia_functions import data_train_test

In [ ]:
from kerastuner.tuners import RandomSearch

Using TensorFlow backend


In [1]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=8, max_value=136, step=8)))
    model.add(keras.layers.Dense(units=hp.Int('units', min_value=4, max_value=32, step=4)))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [2]:
X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)

NameError: name 'data_train_test' is not defined

In [ ]:
tuner = RandomSearch(
    build_model,
    objective = 'val_r2',
    max_trials = 5,
    executions_per_trial = 3,
)

In [ ]:
tuner.search(X_train, y_train, epochs=15, validation_data=(X_test, y_test))